# Exploratory Data Analysis (EDA)

Here I will explore relationships between the different features in my data.

In [30]:
# imports
import pandas as pd

In [32]:
train = pd.read_csv("./data/00_temp_train_data_to_be_removed.csv", index_col=0)
train.head()

,vendor_project,product,vulnerability_name,short_description,required_action,due_date,known_ransomware_campaign_use,notes
837,Zimbra,Collaboration (ZCS),Zimbra Collaboration (ZCS) Arbitrary File Uplo...,Zimbra Collaboration (ZCS) allows an attacker ...,Apply updates per vendor instructions.,2022-11-10,Unknown,https://wiki.zimbra.com/wiki/Security_Center
746,Google,Chromium V8,Google Chromium V8 Out-of-Bounds Write Vulnera...,Google Chromium V8 Engine contains out-of-boun...,Apply updates per vendor instructions.,2022-06-22,Unknown,NaN
1132,SolarWinds,Serv-U,SolarWinds Serv-U Path Traversal Vulnerability,SolarWinds Serv-U contains a path traversal vu...,Apply mitigations per vendor instructions or d...,2024-08-07,Unknown,https://www.solarwinds.com/trust-center/securi...
202,Mozilla,Firefox and Thunderbird,Mozilla Firefox And Thunderbird Use-After-Free...,Mozilla Firefox and Thunderbird contain a race...,Apply updates per vendor instructions.,2022-05-03,Unknown,NaN
547,Rails,Ruby on Rails,Ruby on Rails Directory Traversal Vulnerability,Directory traversal vulnerability in Action Vi...,Apply updates per vendor instructions.,2022-04-15,Unknown,NaN


### Using Train data to find more ways to tidy data


#### Start by looking into vulnerability names

In [33]:
# vulnerability_name value count
train["vulnerability_name"].value_counts()

vulnerability_name
Microsoft Win32k Privilege Escalation Vulnerability                         11
Google Chromium V8 Type Confusion Vulnerability                              7
Microsoft Windows Kernel Privilege Escalation Vulnerability                  7
Adobe Flash Player Use-After-Free Vulnerability                              7
Apple Multiple Products Memory Corruption Vulnerability                      6
                                                                            ..
QNAP NAS File Station Cross-Site Scripting Vulnerability                     1
D-Link and TRENDnet Multiple Devices Remote Code Execution Vulnerability     1
Apple Multiple Products Integer Overflow Vulnerability                       1
SonicWall Email Security Unrestricted Upload of File Vulnerability           1
Google Chromium V8 Heap Buffer Overflow Vulnerability                        1
Name: count, Length: 552, dtype: int64

* I see the vulnerability with the most occurances to be the `Microsoft Win32k Privilege Escalation Vulnerability with count 11`. Let see if we can find more related to this one. 

In [34]:
# vulnerability_name with `Microsoft Win32k` in the name
train.vulnerability_name[train.vulnerability_name.str.contains("Microsoft Win32k")].value_counts()

vulnerability_name
Microsoft Win32k Privilege Escalation Vulnerability    11
Microsoft Win32k Memory Corruption Vulnerability        1
Name: count, dtype: int64

* I see two names with `Microsoft Win32k` in them but one is a `memory curruption vulnerability`. So, let see what that's about

In [35]:
# Hese are apearances of memory curruption in the data
train.vulnerability_name[train.vulnerability_name.str.contains("Memory Corruption")].value_counts()

vulnerability_name
Apple Multiple Products Memory Corruption Vulnerability                         6
Microsoft Internet Explorer Scripting Engine Memory Corruption Vulnerability    4
Microsoft Office Memory Corruption Vulnerability                                3
Apple Multiple Products WebKit Memory Corruption Vulnerability                  3
Microsoft Internet Explorer Memory Corruption Vulnerability                     3
Adobe Flash Player Memory Corruption Vulnerability                              2
Apple iOS Memory Corruption Vulnerability                                       2
Apple Memory Corruption Vulnerability                                           1
Apple iOS, iPadOS, and watchOS Memory Corruption Vulnerability                  1
Google Chromium V8 Memory Corruption Vulnerability                              1
Apple iOS WebKit Memory Corruption Vulnerability                                1
Adobe Acrobat and Reader Universal 3D Memory Corruption Vulnerability          

In [36]:
# Find vetor projects that are responsible for all the memory corruption
train["product"][train.vulnerability_name.str.contains("Memory Corruption")].value_counts()

product
Multiple Products                     9
Internet Explorer                     7
iOS                                   4
Office                                3
Flash Player                          2
iOS and macOS                         1
Win32k                                1
Graphics Component                    1
Excel                                 1
Reader and Acrobat                    1
Internet Explorer Scripting Engine    1
iOS, iPadOS, and watchOS              1
Edge                                  1
Acrobat and Reader                    1
Chromium V8                           1
Edge and Internet Explorer            1
Name: count, dtype: int64

In [37]:
# Find vetor projects that are responsible for all the memory corruption
train.vendor_project[train.vulnerability_name.str.contains("Memory Corruption")].value_counts()

vendor_project
Microsoft    16
Apple        15
Adobe         4
Google        1
Name: count, dtype: int64

* The source of memory corruption is from 4 vendors, but it looks like it's happening on different products of these vendors, so I will go back inthe wrangle file and separate the product from the volunerability and create a additional column for specific vulnerabilities groupd by wording.

**I will do this for all the vulnarabilities I can find. because I suspect there is things that can be grouped together.**

In [38]:
df = pd.read_csv("./data/01_original_clean_no_dummies_train.csv")

In [39]:
df

,Unnamed: 0,vendor_project,product,known_ransomware_campaign_use,vulnerability_type,required_action_new
0,837,Zimbra,Collaboration (ZCS),Unknown,File Upload Vulnerability,Apply Updates
1,746,Google,Chromium V8,Unknown,Out-of-Bounds Write Vulnerability,Apply Updates
2,1132,SolarWinds,Serv-U,Unknown,Path Traversal Vulnerability,Apply Mitigations or Discontinue
3,202,Mozilla,Firefox and Thunderbird,Unknown,Use-After-Free Vulnerability,Apply Updates
4,547,Rails,Ruby on Rails,Unknown,Path Traversal Vulnerability,Apply Updates
...,...,...,...,...,...,...
654,460,Adobe,ColdFusion,Unknown,Sandbox Bypass Vulnerability,Apply Updates
655,312,Hikvision,Security cameras web server,Unknown,Improper Access Control Vulnerability,Apply Updates
656,180,Microsoft,Internet Explorer,Unknown,Memory Corruption Vulnerability,Apply Updates
657,404,Cisco,IOS Software and Cisco IOS XE Software,Unknown,Denial of Service Vulnerability,Apply Updates
